# Data Description

## For this challenge you are given the purchase history of customers across time, along with supporting metadata. Your challenge is to predict what articles each customer will purchase in the 7-day period immediately after the training data ends. Customer who did not make any purchase during that time are excluded from the scoring.

### Files

#### images/ - a folder of images corresponding to each article_id; images are placed in subfolders starting with the first three digits of the article_id; note, not all article_id values have a corresponding image.

#### articles.csv - detailed metadata for each article_id available for purchase

#### customers.csv - metadata for each customer_id in dataset

#### sample_submission.csv - a sample submission file in the correct format

#### transactions_train.csv - the training data, consisting of the purchases each customer for each date, as well as additional information. Duplicate rows correspond to multiple purchases of the same item. Your task is to predict the article_ids each customer will purchase during the 7-day period immediately after the training data period.

#### NOTE: You must make predictions for all customer_id values found in the sample submission. All customers who made purchases during the test period are scored, regardless of whether they had purchase history in the training data.

In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.width',1000)

In [ ]:
df_articles = pd.read_csv("C:/Users/cankemaloglu/Desktop/H&M DAta/articles.csv")
df_transactions_train = pd.read_csv("C:/Users/cankemaloglu/Desktop/H&M DAta/transactions_train.csv")
df_sample_submission = pd.read_csv("C:/Users/cankemaloglu/Desktop/H&M DAta/sample_submission.csv")
df_customer_detail = pd.read_csv("C:/Users/cankemaloglu/Desktop/H&M DAta/customers.csv")

In [18]:
df_customer_detail.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.00,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.00,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.00,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.00,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.00,1.00,ACTIVE,Regularly,52.00,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [19]:
df_sample_submission.head()

,customer_id,prediction
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,0706016001 0706016002 0372860001 0610776002 07...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,0706016001 0706016002 0372860001 0610776002 07...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,0706016001 0706016002 0372860001 0610776002 07...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,0706016001 0706016002 0372860001 0610776002 07...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,0706016001 0706016002 0372860001 0610776002 07...


In [20]:
df_transactions_train.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.05,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.03,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.02,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.02,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.02,2


In [21]:
train = df_transactions_train.reset_index()\
          .merge(df_customer_detail , on = 'customer_id', how='left')
train.head()

,index,t_dat,customer_id,article_id,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.05,2,NaN,NaN,ACTIVE,NONE,24.00,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
1,1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.03,2,NaN,NaN,ACTIVE,NONE,24.00,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
2,2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
3,3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...
4,4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...


In [51]:
train["Active_MembershipStatus_Age"] = train["Active"].astype(str) + "_" + train["club_member_status"] + "_" + train["age"].astype(str)

In [52]:
train.head()

,index,t_dat,customer_id,article_id,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,Active_MembershipStatus_Age
0,0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.05,2,NaN,NaN,ACTIVE,NONE,24.00,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,nan_ACTIVE_24.0
1,1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.03,2,NaN,NaN,ACTIVE,NONE,24.00,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,nan_ACTIVE_24.0
2,2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...,1.0_ACTIVE_32.0
3,3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...,1.0_ACTIVE_32.0
4,4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...,1.0_ACTIVE_32.0


In [62]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31788324 entries, 0 to 31788323
Data columns (total 13 columns):
 #   Column                       Dtype         
---  ------                       -----         
 0   index                        int64         
 1   t_dat                        datetime64[ns]
 2   customer_id                  object        
 3   article_id                   int64         
 4   price                        float64       
 5   sales_channel_id             int64         
 6   FN                           float64       
 7   Active                       float64       
 8   club_member_status           object        
 9   fashion_news_frequency       object        
 10  age                          float64       
 11  postal_code                  object        
 12  Active_MembershipStatus_Age  object        
dtypes: datetime64[ns](1), float64(4), int64(3), object(5)
memory usage: 4.3+ GB


In [59]:
train["age"].agg(["mean","max", "min"])

mean   36.04
max    99.00
min    16.00
Name: age, dtype: float64

In [61]:
train["t_dat"] = pd.to_datetime(train["t_dat"])

In [65]:
def create_date_features(df):
    df['month'] = df.t_dat.dt.month
    df['day_of_month'] = df.t_dat.dt.day
    df['day_of_year'] = df.t_dat.dt.dayofyear
    df['week_of_year'] = df.t_dat.dt.weekofyear
    df['day_of_week'] = df.t_dat.dt.dayofweek
    df['year'] = df.t_dat.dt.year
    df["is_wknd"] = df.t_dat.dt.weekday // 4
    df['is_month_start'] = df.t_dat.dt.is_month_start.astype(int)
    df['is_month_end'] = df.t_dat.dt.is_month_end.astype(int)
    return df

In [66]:
train = create_date_features(train)

C:\Users\CANKEM~1\AppData\Local\Temp/ipykernel_16188/240676347.py:5: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  df['week_of_year'] = df.t_dat.dt.weekofyear


In [67]:
train.head()

,index,t_dat,customer_id,article_id,price,sales_channel_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code,Active_MembershipStatus_Age,month,day_of_month,day_of_year,week_of_year,day_of_week,year,is_wknd,is_month_start,is_month_end
0,0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.05,2,NaN,NaN,ACTIVE,NONE,24.00,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,nan_ACTIVE_24.0,9,20,263,38,3,2018,0,0,0
1,1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.03,2,NaN,NaN,ACTIVE,NONE,24.00,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...,nan_ACTIVE_24.0,9,20,263,38,3,2018,0,0,0
2,2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...,1.0_ACTIVE_32.0,9,20,263,38,3,2018,0,0,0
3,3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...,1.0_ACTIVE_32.0,9,20,263,38,3,2018,0,0,0
4,4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.02,2,1.00,1.00,ACTIVE,Regularly,32.00,8d6f45050876d059c830a0fe63f1a4c022de279bb68ce3...,1.0_ACTIVE_32.0,9,20,263,38,3,2018,0,0,0
